In [1]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import math 
from matplotlib import pyplot as plt


class Node:
    def __init__(self, left=None, right=None, feature=None, threshold=None, value=None, leaf=True):
        self.left = left
        self.right = right
        self.feature = feature
        self.threshold = threshold
        self.value = value #either a split condition (feature or value) or leaf node 
        self.leaf = leaf
        

    def get_n_min(n_min, y):
        return max(1, int((n_min / 100) * len(y)))

class DecisionTree:
    def __init__(self, root=None):
        self.root = root #initalizes the root
        self.n_min = None
        
    def get_n_min(self, n_min, y):
        return max(1, int((n_min / 100) * len(y)))
        
    def entropy(self, y):
        entropy = 0
        size = len(y)
        unique_values, counts = np.unique(y, return_counts=True)
        for i in range(len(unique_values)):
            p_i = counts[i]/size
            if (p_i <= 0):  
                continue
            entropy -= p_i * np.log2(p_i)
        return entropy
    
    def information_gain(self, split_feature, split_threshold, y): #threshold: value at which dataset is split
        size = len(y)
        curr_entropy = self.entropy(y)
        left_indices = np.nonzero(split_feature <= split_threshold)[0]
        right_indices = np.nonzero(split_feature > split_threshold)[0]
        if len(left_indices) == 0 or len(right_indices) == 0: #if empty group, return current value, no more splits
            return 0
        new = ((len(y[left_indices])/size) * self.entropy(y[left_indices])) + ((len(y[right_indices])/size)*self.entropy(y[right_indices]))
        return curr_entropy - new  
    
    def most_occuring_label(self, y): #assigning the most frequent label to leaf Nodes to give its final value/class after all splits
        unique_values, counts = np.unique(y, return_counts=True)
        max_index = np.argmax(counts)
        return unique_values[max_index]
    
    def choose_best_split(self, X, y): #loop through all features to find best root and nodes w highest ig
        best_gain = -1
        best_feature = None
        best_threshold = None
        for i in range(X.shape[1]):
            unique_values = np.unique(X[:,i])
            if len(unique_values)==1: #1 unique value of feature  
                continue
            for values in unique_values:
                ig = self.information_gain(X[:,i], values, y)  
                if (ig>best_gain):
                    best_gain = ig
                    best_feature = i
                    best_threshold = values
        return best_feature, best_threshold
    
    def create_tree(self, X, y, n_min, depth=0):
        minimum = self.get_n_min(n_min, y)  
        if (len(np.unique(y))==1) or (X.shape[0] < minimum):
            return Node(value=self.most_occuring_label(y), leaf=True)  
        best_feature, best_threshold = self.choose_best_split(X, y)
        if best_feature is None: 
            return Node(value=self.most_occuring_label(y), leaf=True)  
        
        left_indices = np.nonzero(X[:, best_feature] <= best_threshold)[0]  
        right_indices = np.nonzero(X[:, best_feature] > best_threshold)[0] 
        X_left = X[left_indices,:]
        X_right = X[right_indices,:]
        y_left = y[left_indices]  
        y_right = y[right_indices]  
        
        left_subtree = self.create_tree(X_left, y_left, n_min)
        right_subtree = self.create_tree(X_right, y_right, n_min)
        
        return Node(left=left_subtree, right=right_subtree, feature=best_feature, 
                   threshold=best_threshold, leaf=False)  
    
    def fit(self, X, y, n_min): #training, actually builds the tree 
        self.n_min = n_min
        self.root = self.create_tree(X, y, self.n_min)
    
    def predict_one(self, node, sample): 
        if node.leaf: #if node leaf
            return node.value 
        elif sample[node.feature] <= node.threshold:
            return self.predict_one(node.left, sample)  
        elif sample[node.feature] > node.threshold:
            return self.predict_one(node.right, sample)  
    
    def predict(self, X):
        y_predictions = [] #initialliy empty
        for i in range(X.shape[0]):
            y_predictions.append(self.predict_one(self.root, X[i]))
        return np.array(y_predictions)  #  for easier comparison



In [2]:
def cross_validate(X, y, n_min=None):
    if n_min is None:
        n_min = [5]
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    n_min_all = {}
    
    for i in n_min:
        train_accuracies = []
        test_accuracies = []
        
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]  
            y_train, y_test = y[train_index], y[test_index]  
            
            tree_train = DecisionTree()
            tree_train.fit(X_train, y_train, i)  
            
            train_preds = tree_train.predict(X_train)  
            train_accuracy = np.mean(train_preds == y_train)
            train_accuracies.append(train_accuracy)
            
            test_preds = tree_train.predict(X_test)  
            test_accuracy = np.mean(test_preds == y_test)
            test_accuracies.append(test_accuracy)
        
        # mean and standard deviation of accuracies for training and testing
        mean_train_accuracy = np.mean(train_accuracies)
        std_train_accuracy = np.std(train_accuracies)
        mean_test_accuracy = np.mean(test_accuracies)
        std_test_accuracy = np.std(test_accuracies)
        
        # results for the current n_min value
        n_min_all[i] = {  
            'mean_train_accuracy': mean_train_accuracy,
            'std_train_accuracy': std_train_accuracy,
            'mean_test_accuracy': mean_test_accuracy,
            'std_test_accuracy': std_test_accuracy
        }
    
    return n_min_all


In [3]:
iris_df = pd.read_csv("iris.csv", names=["sepal_length", "sepal_width", "petal_length", "petal_width", "class"])
with pd.option_context('future.no_silent_downcasting', True):
    iris_df= iris_df.replace({'Iris-setosa':0, 'Iris-versicolor': 1, 'Iris-virginica': 2}).infer_objects()
    iris = iris_df.to_numpy()
    iris_x = iris[:,:-1]
    iris_y = iris[:,-1]

In [5]:
print(cross_validate(iris_x, iris_y, n_min =[5,10,15,20]))
    

{5: {'mean_train_accuracy': 1.0, 'std_train_accuracy': 0.0, 'mean_test_accuracy': 0.9466666666666667, 'std_test_accuracy': 0.0581186525805423}, 10: {'mean_train_accuracy': 1.0, 'std_train_accuracy': 0.0, 'mean_test_accuracy': 0.9466666666666667, 'std_test_accuracy': 0.0581186525805423}, 15: {'mean_train_accuracy': 1.0, 'std_train_accuracy': 0.0, 'mean_test_accuracy': 0.9466666666666667, 'std_test_accuracy': 0.0581186525805423}, 20: {'mean_train_accuracy': 1.0, 'std_train_accuracy': 0.0, 'mean_test_accuracy': 0.9466666666666667, 'std_test_accuracy': 0.0581186525805423}}


In [10]:
spambase_df = pd.read_csv("spambase.csv")
spambase = spambase_df.to_numpy()
spambase_y = spambase[:,-1]
spambase_x = spambase[:,:-1]

In [11]:
print(cross_validate(spambase_x,spambase_y,n_min=[5,10,15,20,25]))

{5: {'mean_train_accuracy': 0.9993961352657005, 'std_train_accuracy': 0.00019474052532119157, 'mean_test_accuracy': 0.9178260869565218, 'std_test_accuracy': 0.014737783701459383}, 10: {'mean_train_accuracy': 0.9993961352657005, 'std_train_accuracy': 0.00019474052532119157, 'mean_test_accuracy': 0.9178260869565218, 'std_test_accuracy': 0.014737783701459383}, 15: {'mean_train_accuracy': 0.9993961352657005, 'std_train_accuracy': 0.00019474052532119157, 'mean_test_accuracy': 0.9178260869565218, 'std_test_accuracy': 0.014737783701459383}, 20: {'mean_train_accuracy': 0.9993961352657005, 'std_train_accuracy': 0.00019474052532119157, 'mean_test_accuracy': 0.9178260869565218, 'std_test_accuracy': 0.014737783701459383}, 25: {'mean_train_accuracy': 0.9993961352657005, 'std_train_accuracy': 0.00019474052532119157, 'mean_test_accuracy': 0.9178260869565218, 'std_test_accuracy': 0.014737783701459383}}



A tree structure is implemented in this model. For this, a Node class is created to represent each node in the decision tree, with attributes for internal nodes, split conidtions, and values. The best splits are determined with the use of calculations of entropy and information gain. Instead of growing a full tree, minimum number of instances (n_min), approach is used where nodes are only split further if they contain more than a certain percentage of the training data. The model is evaluated using 10 fold cross validation across different n_min values 5% to 20% for iris dataset & 5-25% for spambase dataset.
To measure the performance of the model, mean training and testing accuracy along with standard deviations are recorded.